# Find out which echo trains measure which column indexes

In [1]:
from pathlib import Path
import pandas as pd
import sqlite3  # Since we're using sqlite3 as requested.
import twixtools


In [ ]:
dat_path_0003 = Path('E:/01_data/01_prostate_raw_kspace_umcg_p1/0003_patient_umcg_done/kspaces/meas_MID00401_FID373323_t2_tse_traobl_p2_384.dat')
multi_twix = twixtools.read_twix(str(dat_path_0003), parse_geometry=False, parse_data=True)

imaging_data = multi_twix[-1]
print(f"Type of imaging_data: {type(imaging_data)}")
# the type is dict so lets print all keys
print(f"Keys in imaging_data: {imaging_data.keys()}")
# Ther is a 'hdr' key which is a dictionary itself, so lets print its keys too
print(f"Keys in imaging_data['hdr']: {imaging_data['hdr'].keys()}")

print(f"Number of measurements in the last measurement which is the image data: {len(multi_twix[-1])}")

Software version: VD/VE (!?)

Scan  0


100%|██████████| 38.1M/38.1M [00:01<00:00, 34.0MB/s]


Scan  1


100%|██████████| 3.46G/3.46G [01:37<00:00, 38.0MB/s]

Type of imaging_data: <class 'dict'>
Keys in imaging_data: dict_keys(['mdb', 'hdr', 'hdr_str', 'raidfile_hdr'])
Keys in imaging_data['hdr']: dict_keys(['Config', 'Config_raw', 'Dicom', 'Dicom_raw', 'Meas', 'Meas_raw', 'MeasYaps', 'MeasYaps_raw', 'Phoenix', 'Phoenix_raw', 'Spice', 'Spice_raw'])
Number of measurements in the last measurement which is the image data: 4


In [3]:
mdb = multi_twix[-1]['mdb'][0] # first mdb element of last measurement
x   = mdb.data
hdr = mdb.mdh
print(f"Data type of x: {x.dtype}, shape: {x.shape}")
print(f"Data type of hdr: {type(hdr)}")
print(f"hdr = {hdr}")

flags = mdb.get_active_flags()
print(f"Flags: {flags}, with type {type(flags[0])}")

Data type of x: complex64, shape: (20, 768)
Data type of hdr: <class 'twixtools.mdh_def.Scan_header'>
hdr = [123712, 401, 1, 31542297, 19411726, 0, 5384, 0, 0, -2161006, 0, 34078728, 768, 20, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0], 0, 0, 0.0, 0, 0, 0, ['[0.0, -41.1547966003418, -37.76484680175781]', array([ 9.6053678e-01, -2.7815297e-01,  6.8100039e-13,  2.3516768e-12],
      dtype=float32)], array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0], dtype=uint16), array([0, 0, 0, 0], dtype=uint16), 0, 0, 0]
Flags: ['ONLINE', 'SWAPPED', 'NOISEADJSCAN'], with type <class 'str'>


In [ ]:
# read all image data from file
def read_image_data(filename):
    out = list()
    for mdb in twixtools.read_twix(filename)[-1]['mdb']:
        if mdb.is_image_scan():
            out.append(mdb.data)
    return out

dat_path_0003 = Path('E:/01_data/01_prostate_raw_kspace_umcg_p1/0003_patient_umcg_done/kspaces/meas_MID00401_FID373323_t2_tse_traobl_p2_384.dat')
# read all image data from file


mdb = twixtools.read_twix(str(dat_path_0003), parse_geometry=False, parse_data=True)[-1]['mdb'][1000] # 1000th mdb element of last measurement
# Get a list of all attributes and methods of the mdb object
attributes = dir(mdb)
print(attributes)


Software version: VD/VE (!?)

Scan  0


100%|██████████| 38.1M/38.1M [00:00<00:00, 623MB/s]


Scan  1


100%|██████████| 3.46G/3.46G [00:03<00:00, 1.15GB/s]


['IceProgramPara', '_Mdb__get_data', '_Mdb__read_mdh', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__firstlineno__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__static_attributes__', '__str__', '__subclasshook__', '__weakref__', '_get_block_len', '_get_data_len', '_get_dma_len', 'add_flag', 'block_len', 'cAve', 'cEco', 'cLin', 'cPar', 'cPhs', 'cRep', 'cSeg', 'cSet', 'cSlc', 'channel_hdr', 'clear_all_flags', 'convert_to_local', 'data', 'data_len', 'dma_len', 'fid', 'get_active_flags', 'get_flag', 'get_flags', 'is_flag_set', 'is_image_scan', 'mdh', 'mem_pos', 'remove_flag', 'set_flag', 'set_flags', 'set_pmutimestamp', 'set_timestamp', 'set_timestamps', 'update_CRC', 'version_is_ve', 'write_to_file']


In [5]:
def print_mdb_info(mdb):
    """
    Print a summary of relevant information from an mdb object.
    
    The printed information includes:
      - Averaging, line, partition, slice, echo, repetition, and set counters.
      - The shape of the data block and basic statistics.
      - Active MDH flags.
      - Names of the miniheader fields.
      - A truncated view of the channel header.
      - Whether the version is VE.
    """
    print("=== MDB Information Summary ===")
    try:
        print(f"cAve (Average): {mdb.cAve}")
    except AttributeError:
        print("cAve: Not available")
    
    try:
        print(f"cLin (Line index): {mdb.cLin}")
    except AttributeError:
        print("cLin: Not available")
    
    try:
        print(f"cPar (Partition index): {mdb.cPar}")
    except AttributeError:
        print("cPar: Not available")
    
    try:
        # Sometimes stored as cSlc (for slice); might not be present in all data.
        print(f"cSlc (Slice index): {mdb.cSlc}")
    except AttributeError:
        print("cSlc: Not available")
    
    try:
        print(f"cEcho (Echo index): {mdb.cEcho}")
    except AttributeError:
        print("cEcho: Not available")
    
    try:
        print(f"cRep (Repetition index): {mdb.cRep}")
    except AttributeError:
        print("cRep: Not available")
    
    try:
        print(f"cSet (Set index): {mdb.cSet}")
    except AttributeError:
        print("cSet: Not available")
    
    print(f"Data shape: {mdb.data.shape}")
    # Print a basic summary of the data to avoid huge dumps
    print(f"Data stats: min={mdb.data.min()}, max={mdb.data.max()}, mean={mdb.data.mean()}")
    
    try:
        active_flags = mdb.get_active_flags()
    except Exception as e:
        active_flags = f"Error retrieving flags: {e}"
    print(f"Active Flags: {active_flags}")
    
    try:
        mdh_fields = mdb.mdh.dtype.names
    except Exception as e:
        mdh_fields = f"Not available: {e}"
    print(f"Miniheader fields: {mdh_fields}")
    
    try:
        # version_is_ve() returns whether the data is acquired in VE mode.
        print(f"Version is VE: {mdb.version_is_ve()}")
    except Exception as e:
        print("Version check: Not available")
    
    # Channel header may be large; here we print a short truncation.
    chan_hdr_str = str(mdb.channel_hdr)
    print(f"Channel header (first 200 chars): {chan_hdr_str[:200]}")
    
    # cEco is always 0 for some reason on all averages
    # cPar is always 0 for some reason on all averages
    # cPhs is always 0 for some reason on all averages
    # cRep is always 0 for some reason on all averages
    # cSet is always 0 for some reason on all averages
    print(f"cEco is always 0 for some reason on all averages")
    print(f"cPar is always 0 for some reason on all averages")
    print(f"cPhs is always 0 for some reason on all averages")
    print(f"cRep is always 0 for some reason on all averages")
    print(f"cSet is always 0 for some reason on all averages")
    print("=" * 40)

In [ ]:
patients         = ['0003']
avg1, avg2, avg3 = 0, 1, 2
idx_non_img_scan = -1
first_imge_idx   = -1
n_slices         = 0
db_fpath         = Path('databases/master_habrok_20231106_v2.db')
tablename        = 'echo_train_mapping_debug'

# Create the database if it doesn't exist
conn = sqlite3.connect(db_fpath)
c = conn.cursor()
c.execute(f"CREATE TABLE IF NOT EXISTS {tablename} (idx INTEGER PRIMARY KEY, seq_id TEXT, cAve INTEGER, cLin INTEGER, cSeg INTEGER, cSlc INTEGER)")
conn.commit()
conn.close()

for pat in patients:
    print(f"ID of the patient: {pat}")
    # dat_fpath = Path() here we'll load the relevant .dat file.
    acq_data = list()       # we store the ordering of the data only
    for acq_idx, mdb in enumerate(multi_twix[-1]['mdb']):
        if mdb.is_image_scan():
            if first_imge_idx == -1:
                print_mdb_info(mdb)
                print(f"Data shape = {mdb.data.shape}")
                first_imge_idx = acq_idx
            # All attributes of the mdb object: ['IceProgramPara', '_Mdb__get_data', '_Mdb__read_mdh', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__firstlineno__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__static_attributes__', '__str__', '__subclasshook__', '__weakref__', '_get_block_len', '_get_data_len', '_get_dma_len', 'add_flag', 'block_len', 'cAve', 'cEco', 'cLin', 'cPar', 'cPhs', 'cRep', 'cSeg', 'cSet', 'cSlc', 'channel_hdr', 'clear_all_flags', 'convert_to_local', 'data', 'data_len', 'dma_len', 'fid', 'get_active_flags', 'get_flag', 'get_flags', 'is_flag_set', 'is_image_scan', 'mdh', 'mem_pos', 'remove_flag', 'set_flag', 'set_flags', 'set_pmutimestamp', 'set_timestamp', 'set_timestamps', 'update_CRC', 'version_is_ve', 'write_to_file']
            # print(f"cAve: {mdb.cAve}, cEco: {mdb.cEco}, cLin: {mdb.cLin}, cPar: {mdb.cPar}, cPhs: {mdb.cPhs}, cRep: {mdb.cRep}, cSeg: {mdb.cSeg}, cSet: {mdb.cSet}, cSlc: {mdb.cSlc}")
            print(f"idx: {acq_idx}, cAve: {mdb.cAve}, cLin: {mdb.cLin}, cSeg: {mdb.cSeg}, cSlc: {mdb.cSlc}")
            acq_data.append(
                {
                    'idx':    acq_idx,
                    'seq_id': pat,
                    'cAve':   mdb.cAve,
                    'cLin':   mdb.cLin,
                    'cSeg':   mdb.cSeg,
                    'cSlc':   mdb.cSlc,
                }
            )
        else: 
            if acq_idx % 50 == 0:
                print(f"Not an image scan: {acq_idx}")
    
    df = pd.DataFrame(acq_data)
    # now we insert it into the existing table
    conn = sqlite3.connect(db_fpath)
    c = conn.cursor()
    # Insert the data into the database
    for row in acq_data:
        c.execute(f"INSERT OR IGNORE INTO {tablename} (idx, seq_id, cAve, cLin, cSeg, cSlc) VALUES (?, ?, ?, ?, ?, ?)",
                  (row['idx'], row['seq_id'], row['cAve'], row['cLin'], row['cSeg'], row['cSlc']))


    print(f"\n\nNumber of image scans: {len(acq_data)}")


ID of the patient: 0003
Not an image scan: 0
Not an image scan: 50
Not an image scan: 100
Not an image scan: 150
Not an image scan: 200
Not an image scan: 250
Not an image scan: 300
Not an image scan: 350
Not an image scan: 400
Not an image scan: 450
Not an image scan: 500
Not an image scan: 550
Not an image scan: 600
Not an image scan: 650
Not an image scan: 700
Not an image scan: 750
=== MDB Information Summary ===
cAve (Average): 0
cLin (Line index): 535
cPar (Partition index): 0
cSlc (Slice index): 0
cEcho: Not available
cRep (Repetition index): 0
cSet (Set index): 0
Data shape: (20, 768)
Data stats: min=(-2.1080486476421356e-05+9.318115189671516e-06j), max=(2.2576190531253815e-05+5.2263494580984116e-06j), mean=(-3.251404123716384e-08-5.523519064354332e-08j)
Active Flags: ['ONLINE', 'SWAPPED', 'FIRSTSCANINSLICE']
Miniheader fields: Not available: 'Scan_header' object has no attribute 'dtype'
Version check: Not available
Channel header (first 200 chars): [<twixtools.mdh_def.Channel_

# lets do a speed test by reading the mrd from the research drive
\\zkh\appdata\research\Radiology\fastMRI_PCa\03_data\002_processed_pst_ksp_umcg_backup\data\pat_data\0015_ANON9844606

In [11]:
mrd_path = Path(r"\\zkh\appdata\research\Radiology\fastMRI_PCa\03_data\002_processed_pst_ksp_umcg_backup\data\pat_data\0015_ANON9844606\mrds\meas_MID00109_FID379687_t2_tse_traobl_p2_384-out_2.mrd")

multi_twix = twixtools.read_twix(str(mrd_path), parse_geometry=False, parse_data=True)


Software  : VB (!?)

Scan  0


  4%|▍         | 122M/2.81G [00:00<00:17, 169MB/s] 
